# Prediction code sample

- This is the code only run for estern Africa region

## todo:

1.  TODO: split function
    - TODO: 1. feature layer prepare
    - TODO: 2. prediction
    - TODO: 3. post processing
    
2. add geojson and task json with static github url

3. align dea_ml and deafrica_tools avoid duplicates.

4. TODO: local run pipeline script

5. TODO:docker run pipline -> k8s, docker image building test for k8s, add the yaml and other config into the crop-mask repo

6. add one notebook on data prepare for the prediction to run

7. add one notebook to explain configuration and stac files on s3. Explain how the odc-stats OutputProduct fit into the stac_transform function.

8. (?) explain local files indexing. (Or leave it to core team's k8s stack.

In [10]:
import glob
import json
import logging
import math
import os
import os.path as osp
import time

import psutil
from datacube.utils.dask import start_local_dask
from odc.io.cgroups import get_cpu_quota, get_mem_quota
from odc.stats._cli_common import setup_logging

from dea_ml.core.merge_tifs_to_ds import PredictFromFeature
from dea_ml.core.product_feature_config import FeaturePathConfig
from distributed import Client, LocalCluster

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [11]:
import json
from typing import Dict, Tuple

def geofeature_to_tileidx(feature: Dict) -> Tuple[int, int]:
    title = feature["properties"]["title"]
    x_str, y_str = title.split(",")
    return int(x_str), int(y_str)


with open("/home/jovyan/wa/u23/crop-mask/eastern_cropmask/data/s2_tiles_eastern_aez.geojson") as fhin:
    raw = json.load(fhin)
    tile_indicies = [geofeature_to_tileidx(feature) for feature in raw['features']]
    tasks = [f'x{x:+04d}/y{y:+04d}' for x, y in tile_indicies]

In [12]:
tasks[-2:]

['x+047/y+010', 'x+048/y+010']

In [13]:

setup_logging()

_log = logging.getLogger(__name__)

# nthreads = get_max_cpu()
# memory_limit = get_max_mem()

with open("/home/jovyan/wa/u23/notebooks/s2_tiles_eastern_aez_tasks.json") as fhin:
    tasks = json.load(fhin)

output_fld = osp.join(
    FeaturePathConfig.DATA_PATH,
    FeaturePathConfig.product.name,
    FeaturePathConfig.product.version,
)


# manually add tasks
# tasks = ["x+029/y+000/2019-P6M", "x+048/y+010"]

with LocalCluster() as cluster:
    with Client(cluster) as client:
        worker = PredictFromFeature(client=client)
        for task in tasks:
            tile_indx = "/".join(task.split("/")[:2])

            file_prefix = f"{tile_indx}"
            output_path = osp.join(output_fld, file_prefix, "*")
            if glob.glob(output_path):
                _log.warning(f"tile {output_path} is done already. Skipping...")
                continue
            _log.info(f"proessing tiles for task {output_path}.")

            t0 = time.time()
            worker.run(task)
            t1 = time.time()
            wall_time = (t1 - t0) / 60
            _log.info(f"time used {wall_time:.4f}")

SyntaxError: invalid syntax (<ipython-input-13-4ac874733eb0>, line 13)

In [1]:
import sys

sys.path.append('/home/jovyan/wa/u23/crop-mask/dea_ml')

In [2]:
from dea_ml.core.product_feature_config import FeaturePathConfig

In [3]:
cfg = FeaturePathConfig()

In [6]:
cfg.DATA_PATH

'/g/data/u23/data/'

In [7]:
cfg.DATA_PATH = "/tmp"

In [9]:
cfg.DATA_PATH

'/tmp'

In [ ]:
aws s3 sync <local> s3://bucket/<prefix>